In [1]:
from utils import SUPPORTED_MODELS
from transformers import AutoConfig
from optimum.intel.openvino import OVModelForCausalLM
from pathlib import Path
import openvino as ov
from transformers import AutoTokenizer

examine_fp16 = bool(1)  # otherwise bf16

ov_config = {'PERFORMANCE_HINT': 'LATENCY', 'NUM_STREAMS': '1', "CACHE_DIR": ""}

if examine_fp16:
    model_id = "red-pajama-3b-chat"
    models_dir = Path("./")
    model_dir = models_dir / model_id / "FP16"
    # model_dir = models_dir / model_id / "FP16_calibrated"
    # model_dir = models_dir / model_id / "INT8_compressed_weights"
    model_configuration = SUPPORTED_MODELS[model_id]
    # device = "GPU"
    device = "CPU"
else:
    model_dir = Path("/home/devuser/nsavelye/workspace/openvino.genai/llm_bench/python/codegen-2B-multi/pytorch/dldt/FP32")
    model_configuration = {}
    device = "CPU"
    ov_config["INFERENCE_PRECISION_HINT"] = "f32"     # otherwise BF16 is used

core = ov.Core()
tok = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
ov_model = OVModelForCausalLM.from_pretrained(model_dir, device=device, ov_config=ov_config,
                                              config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
                                              trust_remote_code=True)

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


2023-11-21 13:04:54.877665: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-21 13:04:54.879735: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-21 13:04:54.916386: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-21 13:04:54.917281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 13:04:55.487187: W tensorflow/compiler/tf2t

In [3]:
import importlib
import shutil
import partially_upcast_nodes_to_fp32
import model_upcast_utils
import main
importlib.reload(partially_upcast_nodes_to_fp32)
importlib.reload(main)

if examine_fp16:
    example_prompt = "<human>: Which lakes are near Munich?\n<bot>:"
else:
    example_prompt = "# this function implement Fourier transform for input array X"

example_input = ov_model.model, main.get_inputs_for_calibration(
    ov_model, tok, model_configuration, example_prompt)

model = partially_upcast_nodes_to_fp32.partially_upcast_nodes_to_fp32(ov_model.model, example_input, batch_size=50, verbose=True,
                                                                      half_type="fp16" if examine_fp16 else "bf16")
# # model = model_upcast_utils.partially_upcast_nodes_to_fp32(ov_model.model, example_input)
# calibrated_model_dir = Path(f"{model_dir}_calibrated")
# shutil.copytree(model_dir, calibrated_model_dir)
# ov.save_model(model, calibrated_model_dir / "openvino_model.xml")
#
# ov_model.model = model
# ov_model._original_model = model
# ov_model.request = None
# ov_model.compile()

  0%|                                                                                                                                 | 0/4 [00:00<?, ?it/s]


RuntimeError: Exception from src/inference/src/core.cpp:113:
[ GENERAL_ERROR ] Check 'all_devices.size() > idx' failed at src/plugins/proxy/src/plugin.cpp:512:
Cannot get fallback device for index: 0. The total number of found devices is 0



In [3]:
import importlib
import main
importlib.reload(main)

if examine_fp16:
    prompt = "Which lakes are near Munich?"
    generation_kwargs = dict(
        max_new_tokens=100,
        temperature=0.1,
        do_sample=0.1 > 0.0,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.2
    )
else:
    prompt = "# this function implement Fourier transform for input array X"
    generation_kwargs = dict(
        max_new_tokens=100,
        num_beams=1,
        use_cache=True,
    )


# print(run_generate(ov_model, prompt, model_configuration, **generation_kwargs))
for text in main.run_generate(ov_model, tok, prompt, model_configuration, pass_attention_mask=not examine_fp16, **generation_kwargs):
    print(text, end="")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/devuser/nsavelye/venvs/fp16_calibration/lib/python3.8/site-packages/optimum/intel/openvino/modeling_decoder.py:388: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  self.request.start_async(inputs, shared_memory=True)


# this function implement Fourier transform for imput array X
    # and output array Y
    def fft(self, X):
        N = len(X)
        Y = np.zeros(N)
        for i in range(N):
            Y[i] = np.real(np.fft.fft(X[i]))
        return Y

    # this function implement inverse Fourier transform for output array Y
    # and input array X
    def ifft(